### About this Version

After having talked to Herbert und Teresa (12.07.22) we have decided on a new-ish approach on how to tackle the problem with the primitive vector in 2d. This notebook copies part of the old versions, but is not a copy in itself, so that hopefully the result is a bit cleaner.

In [4]:
import random as rd
import numpy as np
import numpy.linalg as la
#import math
from math import gcd
#import matplotlib.pyplot as plt
#from mpl_toolkits.mplot3d import Axes3D

import sympy as sp

In [5]:
def gcd3(a,b,c):
    return gcd(gcd(a,b),c)

def gcdExtended(a, b): 
    # Base Case 
    if a == 0 :  
        return b,0,1

    gcd,x1,y1 = gcdExtended(b%a, a) 

    # Update x and y using results of recursive 
    # call 
    x = y1 - (b//a) * x1 
    y = x1 

    return gcd,x,y

def lcm(a, b):
    return abs(a*b) // gcd(a, b)


In [6]:
def gen_int_basis(d=3, a=5):
    # d is the dimension of the ambient space and the number of resulting vectors
    # a is the maximum length of a resulting vector in the sup norm
    basis = np.zeros((d,d))
    
    det = la.det(basis)
    counter = 0
    while det == 0:
        for i in range(d):
            basis[:,i] = gen_int_vec(d,a) 
        det = la.det(basis)
        counter += 1
        if counter == 100:
            break # raise some error here
            
    return basis

def gen_int_vec(d=3, a=5):
    vec = np.zeros(d)
    for i in range(d):
        vec[i] = rd.randint(-a,a)
    """
    make sure that this does not return the zero vector!
    """
    return vec

### Lemma 3.1 (Parallel Vectors). 
Let $Z \subset \mathbb Z^d$ be a sublattice with $dim Z = d$. Then for every
 $u \in \mathbb Z^d$, there exists a vector $v \in Z$ that is parallel to $u$.

In [7]:
# function that takes basis (which generates sublattice Z) and vector v
# outputs vector u as linear combination of basis elements, which is parallel to v
# also outputs or at least explicitely calculates the coefficients involved

In [8]:
def L31_parallel_vectors(basis,u):
    # function that takes basis (which generates sublattice Z) and vector v
    # outputs vector u as linear combination of basis elements, which is parallel to v
    # also outputs or at least explicitely calculates the coefficients involved
    # input is numpy arrays, output is sympy matrices
    G = basis
    Gi = G**(-1)
    denom_list = []
    p = 1
    for entry in Gi:
        if entry.q not in denom_list:
            denom_list.append(entry.q)
            p *= entry.q
    
    up = p*Gi*u
    v = G*up
    if len(u)==3:
        v = v / gcd3(up[0],up[1],up[2])
        coeff = up / gcd3(up[0],up[1],up[2]) #dividng by gcd of coefficients gives us the shortest parallel vector in the span of the original basis
    if len(u)==2:
        v = v / gcd(up[0],up[1])
        coeff = up / gcd(up[0],up[1])
    
    # to find shortest vector in the span of the new basis (four vectors), 
    # we find out the ratios of the two vectors 
    # get them to integer by multiplying with denominators
    # calculate the gcd of the resulting numbers
    # then divide the gcd (or the vector associated to it) by the denominator again
    for i in range(len(u)):
        if u[i]!=0:
            num1 = v[i]
            num2 = u[i]
            denom = num1.q*num2.q
            new_gcd, a, b = gcdExtended(num1*denom,num2*denom)
            x = new_gcd / sp.sympify(num1.p*num2.q)
            v = v/num1*new_gcd
            coeff = coeff * a
            u_coeff = b
            break
    
    return v, coeff, u_coeff

In [9]:
def primitive(v, coeff): 
    if len(v)==3:
        divider = gcd3(coeff[0],coeff[1],coeff[2])
    elif len(v)==2:
        divider = math.gcd(coeff[0],coeff[1])
    return v/divider, coeff/divider

# Note that, if the primitive vector outputted here agrees with the original u, then we don't need to do anything else

In [10]:
# These are our test-values for repeatability 
# but we can also generate samples using our functions gen_int_basis and gen_int_vec
#----------
basis = sp.Matrix([[1,2,3],[1,0,1],[3,2,0]])
#basis = sp.Matrix([[1,0,0],[0,1,0],[0,0,1]])

v1 = basis[:,0]
v2 = basis[:,1]
v3 = basis[:,2]
#u = sp.Matrix([0,0,2])
u = sp.Matrix([0,1,1])
v4=u
v, coeff, u_coeff = L31_parallel_vectors(basis,u)
#primitive(v,coeff)

In [11]:
print("basis:\n",basis)
print("u:\n",u)
print("v:\n",v)
print(f"""v    = a *  v1   + b *   v2    +  c *   v3    +  d *   v4
double check:
{coeff[0]*basis[:,0] + coeff[1]*basis[:,1]  + coeff[2]*basis[:,2]  + u_coeff*u} = {coeff[0]}*{basis[:,0]} + {coeff[1]} * {basis[:,1]} + {coeff[2]} * {basis[:,2]} + {u_coeff} * {u}""")

basis:
 Matrix([[1, 2, 3], [1, 0, 1], [3, 2, 0]])
u:
 Matrix([[0], [1], [1]])
v:
 Matrix([[0], [1], [1]])
v    = a *  v1   + b *   v2    +  c *   v3    +  d *   v4
double check:
Matrix([[0], [1], [1]]) = 0*Matrix([[1], [1], [3]]) + 0 * Matrix([[2], [0], [2]]) + 0 * Matrix([[3], [1], [0]]) + 1 * Matrix([[0], [1], [1]])


In [12]:
# Next we perform orthogonal projection along v4


def project(projector, projectee):
    return projectee - projectee.dot(projector)/projector.dot(projector) * projector


Pv1 = project(u,v1)
Pv2 = project(u,v2)
Pv3 = project(u,v3)

print(list(Pv1), list(Pv2), list(Pv3))
# at this point, check if one of the Pvi is 0. In that case it is parallel to u and 
# we just need to pick the gcd, which should be v (?)

[1, -1, 1] [2, -1, 1] [3, 1/2, -1/2]


In [18]:

# generally, we will do another projection onto the e2,e3 plane, so that we have 2d coordinates again
# however, if our vector v4, with which we have done the original orthogonal projection, lies in this plane
# then we cannot do this, because we will lose information
# so we need to check if v4 is orthogonal to e1
# if that is the case, we want to project onto the e1,e3 plane
# however, then we need to check if v4 is orthogonal to e2
# if also that is the case, than v4 is parallel to e3
# thus we can project onto e1,e2
e1 = sp.Matrix([1,0,0])
e2 = sp.Matrix([0,1,0])
e3 = sp.Matrix([0,0,1])


def skip_coord(vec,n=1):
    ind=n-1
    if n == 1:
        new_vec = [vec[ind+1],vec[ind+2]]
    elif n == 2:
        new_vec = [vec[ind-1],vec[ind+1]]
    elif n == 3:
        new_vec = [vec[ind-2],vec[ind-1]]
    # Note that this not very elegant approach is because I don't want the coordinates to be permuted cyclically,
    # but instead just want to skip the given entry
    
    #new_vec = [vec[(ind+1)%3],vec[(ind+2)%3]]
    return sp.Matrix(new_vec)


Pv = sp.Matrix([Pv1[:],Pv2[:],Pv3[:]]).T

def skip_corr_coord(Pv):
    if v4.dot(e1) == 0: # v4 is orthogonal to e1
        if v4.dot(e2) == 0: # v4 is orthgonal to e2
            #project onto the e1, e2 plane
            Pv1_2 = skip_coord(Pv[:,0],3) # the subscript 2 stands for 2-coordinates
            Pv2_2 = skip_coord(Pv[:,1],3)
            Pv3_2 = skip_coord(Pv[:,2],3)
            pass
        else:
            #project the Pvi onto the e1, e3 plane
            Pv1_2 = skip_coord(Pv[:,0],2)
            Pv2_2 = skip_coord(Pv[:,1],2)
            Pv3_2 = skip_coord(Pv[:,2],2)
            pass
    else:
        #project the Pvi onto e2, e3 plane
        Pv1_2 = skip_coord(Pv[:,0],1)
        Pv2_2 = skip_coord(Pv[:,1],1)
        Pv3_2 = skip_coord(Pv[:,2],1)
        pass
    
    return sp.Matrix([Pv1_2[:],Pv2_2[:],Pv3_2[:]]).T


In [20]:
def lcm_mx(M):
        """
        Takes sympy 2x2 matrix with rational coefficients and returns lcm of all denominators 
        """
        lcm_denom = M[0,0].q
        for entry in M[:]:
            lcm_denom = lcm(lcm_denom,entry.q)
        return lcm_denom


# now that we have our 2-dimensional vectors,
# we want to find a linear dependent vector
# once we have that, we want to find the primitive, as we did before
def proj2induct(Pv):
    Pv_2 = skip_corr_coord(Pv)

    # search for vector to make new "extra"
    for i in range(3):
        ind = [(i+1)%3,(i+2)%3]
        if Pv_2[:,ind].det() != 0:
            print(f"Pv{i+1}_2 is new extra vector")
            break
        elif Pv_2[:,ind].det() != 0:
            print(f"Pv{i+1}_2 is new extra vector")
            break
        elif Pv_2[:,ind].det() != 0:
            print(f"Pv{i+1}_2 is new extra vector")
            break
        else:
            print("something went wrong!")


    # making stuff integer
    # determine lcm of denomiators
    lcm_denom = lcm_mx(Pv_2)
    print("new basis")
    sp.pprint(Pv_2[:,ind])

    print("new extra vector")
    sp.pprint(Pv_2[:,i])
    new_primitive, new_coeff, new_coeff_u = L31_parallel_vectors(Pv_2[:,ind]*lcm_denom,Pv_2[:,i]*lcm_denom)
    new_primitive /= lcm_denom

    print("new primitive")
    sp.pprint(new_primitive)
    print("new coefficients:",list(new_coeff)+[new_coeff_u])
    
    return Pv_2[:,ind], Pv_2[:,i]

Pv_2, new_primitive = proj2induct(Pv)

Pv1_2 is new extra vector
new basis
⎡2   3  ⎤
⎢       ⎥
⎣1  -1/2⎦
new extra vector
⎡1⎤
⎢ ⎥
⎣1⎦
new primitive
⎡1⎤
⎢ ⎥
⎣1⎦
new coefficients: [0, 0, 1]


In [15]:
# now we go one layer deeper
# i.e. project to the line orthogonal to new_primitive
# we will then be left with two vectors, which are still in 2-dim representation
# however, in this case, I think, we can just take the gcd of one component
# because they HAVE to be collinear anyways
# this is then the basis for the new 1d lattice

# having that, we can retrace our steps in the other induction direction
# that is, take the 2d preimage of this new vector
# then undo the component projection (how?)
# then look at the integer basis proof to get a basis for the 2d lattice 
# (maybe even before undoing the component projection, if that is easier)
# once you have the basis of the 2d vectors (in their 3d form)
# search for their preimages, i.e. 3d vectors in their 3d form
# then again do the integer basis proof process
# then we SHOULD have our basis

In [16]:
# now we go one layer deeper
# i.e. project to the line orthogonal to new_primitive
# we will then be left with two vectors, which are still in 2-dim representation
PPv = sp.Matrix([project(new_primitive,Pv_2[:,0]).T, project(new_primitive,Pv_2[:,1]).T]).T
sp.pprint(Pv_2)
sp.pprint(PPv)

⎡2   3  ⎤
⎢       ⎥
⎣1  -1/2⎦
⎡1/2   7/4 ⎤
⎢          ⎥
⎣-1/2  -7/4⎦


In [17]:
# look for a non-zero coordinate
def gcd_of_2dvecs(vec_mx):
    for i in range(2):
        if PPv[i,0]!=0:
            break
    denoms = lcm_mx(PPv[i,:])

    a = PPv[i,0]*denoms
    b = PPv[i,1]*denoms
    a,b
    c, x,y = gcdExtended(a,b)
    new_1d_basis = x*PPv[:,0]+y*PPv[:,1]
    return new_1d_basis, x,y

In [92]:
# to go back to 2d (up one induction step)

Matrix([
[2],
[3]])